In [2]:
import tweepy                   
import csv
from datetime import date
from datetime import datetime
import time

## List ids of interest:

Members of congress 34179516

New members of congress 816275409931210752

POTUS 1184094696232144897

Govenors 7560205

US representatives 6196793

Senators 4244910

In [3]:
consumer_key = "<consumer key>"
consumer_secret = "<consumer secret key>"
access_key = "<access key>"
access_secret = "<access secret key>"

In [13]:
# Load Twitter API secrets from an external JSON file
api_key = consumer_key
api_secret_key = consumer_secret
access_token = access_key
access_token_secret = access_secret
# Connect to Twitter API using the secrets
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [14]:
api

In [15]:
path = '/Users/a307/Desktop/twitter_covid/'

In [22]:
def get_all_tweets(screen_name):
    # initialize a list to hold all the Tweets
    alltweets = []
    # make initial request for most recent tweets 
    # (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    # save most recent tweets
    alltweets.extend(new_tweets)
    # save the id of the oldest tweet less one to avoid duplication
    oldest = alltweets[-1].id - 1
    # keep grabbing tweets until there are no tweets left
    while len(new_tweets) > 0:
        print("getting tweets before %s" % (oldest))
        # all subsequent requests use the max_id param to prevent
        # duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
        # save most recent tweets
        alltweets.extend(new_tweets)
        # update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        print("...%s tweets downloaded so far" % (len(alltweets)))
    ### END OF WHILE LOOP ###
    # transform the tweepy tweets into a 2D array that will 
    # populate the csv
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text, tweet.favorite_count,tweet.in_reply_to_screen_name, tweet.retweeted] for tweet in alltweets]
    # write the csv
    with open(path + '%s_tweets.csv' % screen_name, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["id","created_at","text","likes","in reply to","retweeted"])
        writer.writerows(outtweets)
    pass

In [23]:
def get_list_members(api, list_id):
    members = []
    # without this you only get the first 20 list members
    for page in tweepy.Cursor(api.list_members, list_id=list_id).items():
        members.append(page)
    # create a list containing all usernames
    return [ m.screen_name for m in members ]


In [24]:
members = get_list_members(api, 34179516)

In [25]:
len(members)

565

In [27]:
# save tweets of people in a list
for member in members:
    try:
        get_all_tweets(member)
    except Exception as ex:
        print(ex)